In [ ]:
import os
from utils import *
import optuna

In [ ]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [ ]:
study_name = "ImplicitALS"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from implicit.gpu.als import AlternatingLeastSquares as IALSRecommender
from implicit.evaluation import ranking_metrics_at_k

evaluator_validation = EvaluatorHoldout(data_val, cutoff_list=[10])

def objective(trial):
    num_factors = trial.suggest_int('num_factors', 1, 500)
    reg = trial.suggest_float('reg', 1e-5, 1, log=True)
    alpha = trial.suggest_float('alpha', 1e-7, 100, log=True)
    iterations = trial.suggest_categorical('iterations', [10, 20, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])
    
    recommender = IALSRecommender(
        factors=num_factors,
        regularization=reg,
        alpha=alpha,
        iterations=iterations,
        calculate_training_loss=False,
    )
    recommender.fit(
        data_train,
        show_progress=False,
    )
    
    return ranking_metrics_at_k(recommender, data_train, data_val, K=10, show_progress=False)['map']

study.optimize(objective, n_trials=300)

In [ ]:
study.best_params

In [ ]:
from implicit.gpu.als import AlternatingLeastSquares as IALSRecommender

recommender = IALSRecommender(
        factors=study.best_params['num_factors'],
        regularization=study.best_params['reg'],
        alpha=study.best_params['alpha'],
        iterations=study.best_params['iterations'],
        calculate_training_loss=False,
    )
recommender.fit(
    data_train,
    show_progress=True,
)

In [ ]:
from implicit.evaluation import ranking_metrics_at_k

ranking_metrics_at_k(recommender, data_train, data_val, K=10, show_progress=False)

In [ ]:
recommender.recommend(1, data_train[1], N=10, filter_already_liked_items=True)[0]

In [ ]:
def submission2(recommender, users, usermap, itemmap, data_train):
    
    toppoprecommender = TopPop(data_train)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            try:
                item_list = list(recommender.recommend(user_id, data_train[user_id], N=10, filter_already_liked_items=True)[0])
                item_list = [itemmap[i] for i in item_list]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
            except IndexError:
                item_list = toppoprecommender.recommend(0, cutoff=10)
                item_list = [itemmap[i] for i in item_list]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)